# Improving document retrieval

## load and format test cases

In [1]:
import re

In [2]:
with open('../data/test_cases.txt') as f:
    test_cases_raw = f.readlines()

In [3]:
test_cases_raw = "".join(test_cases_raw[0:-1:2]).split('问：')[1:]
test_cases_raw[:3]

['什么是我国第一部编年史著作？\n答：《左传》。\n资料：附：《左传》是我国第一部编年史著作。\n',
 '什么是我国第一部编年国别史？\n答：《国语》。\n资料：附：《国语》是我国第一部编年国别史。\n',
 '“寡人之于国也”下一句是什么？来自哪里？\n答：“寡人之于国也”下一句是“尽心焉耳矣”。这个句子来自《孟子》。\n资料：梁惠王曰：“寡人之于国也，尽心焉耳矣。河内凶，则移其民于河东，移其粟于河内；河东凶亦然。察邻国之政，无如寡人之用心者。邻国之民不加少，寡人之民不加多，何也？”\n资料：《寡人之于国也》（孟子）\n']

In [4]:
test_cases_raw = [re.split(r'\n答：|\n资料：', x) for x in test_cases_raw]
test_cases_raw[:3]

[['什么是我国第一部编年史著作？', '《左传》。', '附：《左传》是我国第一部编年史著作。\n'],
 ['什么是我国第一部编年国别史？', '《国语》。', '附：《国语》是我国第一部编年国别史。\n'],
 ['“寡人之于国也”下一句是什么？来自哪里？',
  '“寡人之于国也”下一句是“尽心焉耳矣”。这个句子来自《孟子》。',
  '梁惠王曰：“寡人之于国也，尽心焉耳矣。河内凶，则移其民于河东，移其粟于河内；河东凶亦然。察邻国之政，无如寡人之用心者。邻国之民不加少，寡人之民不加多，何也？”',
  '《寡人之于国也》（孟子）\n']]

In [5]:
test_cases = [{'q': x[0], 'a': x[1], 'docs': x[2:]} for x in test_cases_raw]
test_cases[:3]

[{'q': '什么是我国第一部编年史著作？', 'a': '《左传》。', 'docs': ['附：《左传》是我国第一部编年史著作。\n']},
 {'q': '什么是我国第一部编年国别史？', 'a': '《国语》。', 'docs': ['附：《国语》是我国第一部编年国别史。\n']},
 {'q': '“寡人之于国也”下一句是什么？来自哪里？',
  'a': '“寡人之于国也”下一句是“尽心焉耳矣”。这个句子来自《孟子》。',
  'docs': ['梁惠王曰：“寡人之于国也，尽心焉耳矣。河内凶，则移其民于河东，移其粟于河内；河东凶亦然。察邻国之政，无如寡人之用心者。邻国之民不加少，寡人之民不加多，何也？”',
   '《寡人之于国也》（孟子）\n']}]

In [6]:
test_cases[-3:-1]

[{'q': '“我相信某一天，在某个人的生命中，他头顶天空中所有云朵的形状将会片刻地与他相似。”出自哪里？',
  'a': '原文来自“我们生活的故事 【美】马克·斯特兰德”中《纪念碑》的节选。',
  'docs': ['《纪念碑》（节选）\n']},
 {'q': '有一个诗人曾经写过类似“有人把国家带进沟里，但这些人说统治是一门艺术”的话。请找到出处。',
  'a': '在“致后代 【德】贝托尔特·布莱希特”的“那些把肉从桌上拿走的人【1937】”中，作者写道：\\n那些把肉从桌上拿走的人\\n教导人们满足。\\n那些获得进贡的人\\n要求人们牺牲。\\n那些吃饱喝够的人向饥饿者\\n描绘将来的美好时代。\\n那些把国家带到深渊里的人\\n说统治太难，普通人\\n不能胜任。',
  'docs': ['那些把肉从桌上拿走的人\\n教导人们满足。\\n那些获得进贡的人\\n要求人们牺牲。\\n那些吃饱喝够的人向饥饿者\\n描绘将来的美好时代。\\n那些把国家带到深渊里的人\\n说统治太难，普通人\\n不能胜任。',
   '那些把肉从桌上拿走的人【1937】',
   '致后代 【德】贝托尔特·布莱希特\n']}]

## load Notion DB

In [7]:
from langchain_community.document_loaders import NotionDBLoader
import tomllib

In [15]:
with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

with open('../.notion_databases.toml', 'rb') as f:
    _DATABASES_NOTION = tomllib.load(f)

In [16]:
_DATABASES_NOTION

{'写作': '8b81c07089344d8d95ddba22bafb8c12',
 '笔记（非文学）': '7443174d151342458fb7e47acdbb0c64',
 '读书笔记（文学）': '808e50735cce4c66a151f3e8b79c8d9d',
 '格言小集': '04d75f24b9544f3f8b220d5e7d87b7c3'}

In [55]:
%%time

def load_notion_dbs(dbs, id):
    loader = NotionDBLoader(
        integration_token=_TOKENS['notion'],
        database_id=dbs[id],
        request_timeout_sec=150,  # optional, defaults to 10
    )
    return loader.load()

docs = dict()

for k in _DATABASES_NOTION:
    docs[k] = load_notion_dbs(_DATABASES_NOTION, k)

HTTPError: 400 Client Error: Bad Request for url: https://api.notion.com/v1/databases/8b81c07089344d8d95ddba22bafb8c12/query

## Testing ideas


* langchain.text_splitter w/ params
* langchain_community.embeddings
* langchain_community.vectorstores
* retrieval methods: cos/dot; llm-aided; MMR (Maximum marginal relevance)